## 深層生成モデルの学習方法はモデルに依存しない
### 目的関数と最適化アルゴリズムが独立に設定・学習できる枠組みが必要
- 深層生成モデルでは，モデルに依存せずに勾配降下法で学習する
    - 通常のDNNの学習と同様，様々なモデルに対して，任意の最適化アルゴリズムを用いる
    - 従来の生成モデルのように，モデルに応じて決まった学習アルゴリズムが選択されることはない
        - (どういう意図の列挙？): ギブスサンプリング，EMアルゴリズム，変分推論
- なお，深層生成モデルはパラメータについて最尤推定(パラメータの分布は考えない)
    - この点で，深層生成モデルはベイズ的ニューラルネットワークとは異なる  
<img src='tutorial_figs/PixyzAPI.png'>

## 目的関数と最適化アルゴリズムが独立に設定できる枠組み(Model API)
- Model API document: https://docs.pixyz.io/en/v0.0.4/models.html  

ここでは定義した確率分布と目的関数を受け取り，モデルの学習を行う流れを確認する

In [38]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

batch_size = 256
seed = 1
torch.manual_seed(seed)

In [39]:
# MNIST datasetの準備
root = 'data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1))])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

### 確率分布の定義

In [48]:
from pixyz.distributions import Normal, Bernoulli

x_dim = 784
z_dim = 64

# inference model q(z|x)
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x"], var=["z"], name="q")

        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}

    
# generative model p(x|z)    
class Generator(Bernoulli):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"], name="p")

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {"probs": torch.sigmoid(self.fc3(h))}
    
gen_ber_x__z = Generator().to(device)
infer_nor_z__x = Inference().to(device)

prior_nor_z = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.),
               var=["z"], features_shape=[z_dim], name="p_{prior}").to(device)

### Lossの定義

In [49]:
# Lossの定義
from pixyz.losses import LogProb
from pixyz.losses import StochasticReconstructionLoss
from pixyz.losses import Expectation as E
from pixyz.losses import KullbackLeibler
from pixyz.utils import print_latex

# 対数尤度
logprob_gen_x__z = LogProb(gen_ber_x__z)

# 期待値E
E_infer_z__x_logprob_gen_x__z = E(infer_nor_z__x, logprob_gen_x__z)

# KLダイバージェンス
KL_infer_nor_z__x_prior_nor_z = KullbackLeibler(infer_nor_z__x, prior_nor_z)

# Lossの引き算
total_loss = KL_infer_nor_z__x_prior_nor_z - E_infer_z__x_logprob_gen_x__z

# Lossのmean
total_loss = total_loss.mean()


# Lossの確認
print_latex(total_loss)

<IPython.core.display.Math object>

In [46]:
kl = KullbackLeibler(infer_nor_z__x, prior_nor_z)
reconst = StochasticReconstructionLoss(infer_nor_z__x, gen_ber_x__z)
vae_loss = (kl + reconst).mean()
print_latex(vae_loss)

<IPython.core.display.Math object>

### ModelAPIに確率分布とLossを渡し，最適化アルゴリズムを設定する

pixyz.modelsのModelを呼び出して使用
主な引数はloss, distributions, optimizer, optimzer_paramsで，それぞれには以下のように格納します
- loss: pixyz.lossesを使用して定義した目的関数のLossを格納
- distributions: pixyz.distributionを使用して定義した，学習を行う確率分布を格納
- optimizer, optimizer_params: 最適化アルゴリズム，そのパラメータを格納  

For more details about Model: https://docs.pixyz.io/en/v0.0.4/_modules/pixyz/models/model.html#Model

In [50]:
from pixyz.models import Model
from torch import optim

optimizer = optim.Adam
optimizer_params = {'lr': 1e-3}

vae_model = Model(loss=total_loss, 
                     distributions=[gen_ber_x__z, infer_nor_z__x],
                     optimizer=optimizer,
                     optimizer_params=optimizer_params
                    )

以上でModelの定義が完了した
目的関数の設定と，最適化アルゴリズムの設定が独立に行えたことを確認できた
次に実際にtrainメソッドについて確認し実際に学習を行う  
Model Classのtrainメソッドでは以下の処理を行なっている  
source code: https://docs.pixyz.io/en/v0.0.4/_modules/pixyz/models/model.html#Model.train
1. 観測データであるxを受け取り(.train({"x": x}))
2. Lossを計算し
3. 1stepパラメーターの更新を行い
4. Lossを出力  

```python
def train(self, train_x={}, **kwargs):
        self.distributions.train()

        self.optimizer.zero_grad()
        loss = self.loss_cls.estimate(train_x, **kwargs)

        # backprop
        loss.backward()

        # update params
        self.optimizer.step()

        return loss
```

In [52]:
epoch_loss = []
for epoch in range(3):
    train_loss = 0
    for x, _ in train_loader:
        x = x.to(device)
        loss = vae_model.train({"x": x})
        train_loss += loss
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: ', train_loss)
    epoch_loss.append(train_loss)

Epoch:  tensor(181.7224, grad_fn=<DivBackward0>)
Epoch:  tensor(139.9781, grad_fn=<DivBackward0>)
Epoch:  tensor(124.6789, grad_fn=<DivBackward0>)


以上で学習を行えることを確認した  
Pixyzでは高度なModelAPIとしてVAE, GAN Modelを用意しており，ただ入力データの変更やDNNのネットワークアーキテクチャーを変更したいだけの場合は高度なModel APIを使用することで簡単に実装することができる

## 高度なModel APIの使用
- Pre-implementation models: https://docs.pixyz.io/en/v0.0.4/models.html#pre-implementation-models